#**Protpardelle-1c**

Code: [GitHub](https://github.com/ProteinDesignLab/protpardelle-1c/)  
Preprint: [bioRxiv](https://www.biorxiv.org/content/10.1101/2025.08.18.670959v2)

Note: the Colab is intended for lightweight sampling. For ESMFold self-consistency and production runs, we recommend setting up the environment on a cluster.

# Install

In [ ]:
%%bash

# Install environment
git clone https://github.com/ProteinDesignLab/protpardelle-1c.git
cd protpardelle-1c
uv pip install -qq -e .
cd /content/
uv pip install -qq py3Dmol

# Download Protpardelle-1c weights
apt-get install aria2 -qq
aria2c -x16 -s16 -o protpardelle-1c.tar.gz "https://zenodo.org/records/16817230/files/protpardelle-1c.tar.gz?download=1"
tar -xzvf protpardelle-1c.tar.gz --strip-components=1
rm protpardelle-1c.tar.gz

# Download ESMFold weights
mkdir -p model_params/ESMFold
hf download facebook/esmfold_v1 --local-dir model_params/ESMFold

# Download ProteinMPNN weights
mkdir -p model_params/ProteinMPNN
tmp="$(mktemp -d)"
repo_url="https://github.com/dauparas/ProteinMPNN.git"
branch="main"
folder="vanilla_model_weights"

git_ver="$(git --version | awk '{print $3}')"
IFS=. read -r M m p <<<"$git_ver"
: "${m:=0}"
: "${p:=0}"
# Check Git version >= 2.25.0
if ((M > 2 || (M == 2 && (m > 25 || (m == 25 && p >= 0))))); then
    git clone --depth=1 --filter=tree:0 --sparse "$repo_url" "$tmp"
    git -C "$tmp" sparse-checkout set "$folder"
    git -C "$tmp" checkout "$branch"
else
    git clone --depth=1 --single-branch --branch "$branch" "$repo_url" "$tmp" \
        || git clone --depth=1 "$repo_url" "$tmp"
fi

mv "$tmp/$folder" model_params/ProteinMPNN/
rm -rf "$tmp"

# Install Foldseek
wget https://mmseqs.com/foldseek/foldseek-linux-gpu.tar.gz; tar xvfz foldseek-linux-gpu.tar.gz; export PATH=$(pwd)/foldseek/bin/:$PATH

# Move weights to default path
mv /content/model_params /content/protpardelle-1c/

In [ ]:
import os
os.environ["FOLDSEEK_PATH"] = "/content/foldseek/bin/foldseek"

# Sampling Examples

Outputs are saved under `/content/protpardelle-1c/results` by default.

In [ ]:
%%capture
# Unconditional sampling
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/00_unconditional.yaml --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# Partial diffusion
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/01_partial_diffusion.yaml --motif-dir /content/protpardelle-1c/examples/motifs/nanobody --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# Motif scaffolding
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/02_motif_scaffolding.yaml --motif-dir /content/protpardelle-1c/examples/motifs/nanobody --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# MotifBench benchmark
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/03_motifbench.yaml --motif-dir /content/protpardelle-1c/examples/motifs/motif_bench --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# BindCraft benchmark
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/04_bindcraft.yaml --motif-dir /content/protpardelle-1c/examples/motifs/bindcraft/ --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# Multichain
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/05_multichain.yaml --motif-dir /content/protpardelle-1c/examples/motifs/nanobody/ --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# Thread in a new sequence given a backbone: experimental feature
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/06_fastrelax.yaml --motif-dir /content/protpardelle-1c/examples/motifs/nanobody/ --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# Backbone-only La-Proteina / RFdiffusion benchmark
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/07_rfdiffusion.yaml --motif-dir /content/protpardelle-1c/examples/motifs/rfdiffusion/ --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# All-atom La-Proteina / RFdiffusion benchmark
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/08_rfdiffusion_allatom.yaml --motif-dir /content/protpardelle-1c/examples/motifs/rfdiffusion/ --num-samples 8 --num-mpnn-seqs 0


In [ ]:
%%capture
# Generate structure conditioned on sequence: experimental feature, useful as a decoy generator
!python -m protpardelle.sample /content/protpardelle-1c/examples/sampling/09_structure_prediction.yaml --motif-dir /content/protpardelle-1c/examples/motifs/nanobody/ --num-samples 8 --num-mpnn-seqs 0

# Visualize

Code adapted from Sergey Ovchinnikov's [ESMFold Colab](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/ESMFold.ipynb).

In [ ]:
#@title {run: "auto"}
import py3Dmol
from string import ascii_uppercase, ascii_lowercase

pymol_color_list = ["#33ff33","#00ffff","#ff33cc","#ffff00","#ff9999","#e5e5e5","#7f7fff","#ff7f00",
                    "#7fff7f","#199999","#ff007f","#ffdd5e","#8c3f99","#b2b2b2","#007fff","#c4b200",
                    "#8cb266","#00bfbf","#b27f7f","#fcd1a5","#ff7f7f","#ffbfdd","#7fffff","#ffff7f",
                    "#00ff7f","#337fcc","#d8337f","#bfff3f","#ff7fff","#d8d8ff","#3fffbf","#b78c4c",
                    "#339933","#66b2b2","#ba8c84","#84bf00","#b24c66","#7f7f7f","#3f3fa5","#a5512b"]
alphabet_list = list(ascii_uppercase+ascii_lowercase)

def show_pdb(pdb_str, show_sidechains=False, show_mainchains=False,
             color="rainbow", chains=None, vmin=50, vmax=90,
             size=(800,480), hbondCutoff=4.0,
             animate=False):

  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=size[0], height=size[1])
  if animate:
    view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  else:
    view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  if color == "pLDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                  {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  view.zoomTo()
  if animate: view.animate()
  return view

pdb_path = "" #@param {type:"string"}
color = "rainbow" #@param ["rainbow", "chain"]
show_sidechains = True #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}


pdb_str = []
chains = set()
with open(pdb_path, 'r') as fp:
  for line in fp.readlines():
    if line.startswith(("ATOM", "HETATM")) and len(line) >= 22:
        chain_id = line[21]  # Column 22 in PDB format (0-based index 21)
        chains.add(chain_id)
        pdb_str.append(line.strip())
pdb_str = '\n'.join(pdb_str)
num_chains = len(chains)


show_pdb(pdb_str, color=color,
         show_sidechains=show_sidechains,
         show_mainchains=show_mainchains,
         chains=num_chains).show()